# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

client = AzureOpenAI(
        azure_endpoint = endpoint, 
        api_key=api_key,
        api_version="2024-02-01"
    )

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [5]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations

--------------------------------------------------------------------------------

Max Tokens: 32

The best pet is subjective and can vary based on individual preferences, lifestyles, and living situations. Some people love dogs for their companionship and loyalty, while others prefer

--------------------------------------------------------------------------------

Max Tokens: 60

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options, along with their benefits:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require exercise and socialization.

2.

--------------------------------------------------------------------------------

Max Tokens: 100

The best pet can vary greatly depending on individual preferences, lifestyles, and l

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [6]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=60,
                seed=SEED,
                temperature=temperature
            )
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=60,
                temperature=temperature
            )
        print(response.choices[0].message.content)

In [7]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

The best pet is often subjective and depends on individual preferences and lifestyles. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families.
2. **Cats**: Independent and low-maintenance, cats are perfect for people
The best pet is subjective and depends on individual preferences, lifestyles, and circumstances. Some people might find dogs to be the best pets because of their loyalty and companionship, while others might prefer cats for their independence and low maintenance. Small animals like hamsters, rabbits, or guinea pigs can be great for
The best pet can vary greatly depending on individual preferences, lifestyle, and needs. Some people might say dogs are the best pets because of their loyalty and companionship. Others might prefer cats for their independence and low-maintenance nature. Small animals like hamsters or guinea pigs can be great for those with limited
The best pet is often subjective

In [ ]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [9]:
def call_openai_with_n(num_times, prompt, temperature=0.75, n=1):
    for i in range(num_times):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=60,
            temperature=temperature,
            n=n
        )
        
        # Imprimir todas las completions generadas
        for index, c in enumerate(response.choices):
            print(f"Completion {index + 1}: {c.message.content}")


In [10]:
prompt = "Describe the benefits of using AI in education."
call_openai_with_n(1, prompt, temperature=0.9, n=3)

Completion 1: The integration of artificial intelligence (AI) in education offers numerous benefits that enhance teaching, learning, and administrative processes. Here are some key advantages:

1. **Personalized Learning**: AI can tailor educational experiences to meet the individual needs of students. By analyzing learning patterns, strengths, and weaknesses,
Completion 2: Artificial Intelligence (AI) is transforming education in numerous ways, offering various benefits that enhance teaching and learning experiences. Here are some of the significant advantages:

1. **Personalized Learning**: AI can tailor educational experiences to meet individual students' needs, preferences, and learning paces. By analyzing performance
Completion 3: The integration of AI in education offers a multitude of benefits that enhance the learning experience, streamline administrative tasks, and foster personalized education. Here are some key advantages:

1. **Personalized Learning**: AI can analyze indivi

# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [12]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": "The best pet is a"}],
        max_tokens=60,
        presence_penalty=presence_penalty
    )
    
    # Retornar la respuesta generada
    return response.choices[0].message.content

# Generar respuestas con diferentes valores de presence_penalty
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Presence Penalty: 0

The best pet is often subjective and can vary based on individual preferences, lifestyles, and living situations. Some people may find dogs to be the best pets due to their loyalty and companionship, while others might prefer cats for their independence and low maintenance. Small pets like hamsters, guinea pigs, or rabbits

--------------------------------------------------------------------------------

Presence Penalty: 0.5

The best pet can vary from person to person, depending on individual preferences, lifestyles, and living situations. Some popular options include:

1. **Dogs**: Known for their loyalty, companionship, and ability to bond closely with humans. They require regular exercise and attention.

2. **Cats**:

--------------------------------------------------------------------------------

Presence Penalty: 1.0

The best pet can vary depending on individual preferences, lifestyles, and needs. Here are a few popular options:

1. **Dogs** - Known for th

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.

## Exercise 1: Exploring 'top_p' Parameter
### Description:
- The 'top_p' parameter controls nucleus sampling. Lower values make the output more focused by limiting the pool of tokens.

### Task:
- Write a function that calls the model with different 'top_p' values and observe how the output changes.


In [16]:
def call_openai_with_top_p(top_p):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": "Describe the benefits of exercise."}],
        max_tokens=60,
        top_p=top_p
    )
    
    # Retornar la respuesta generada
    return response.choices[0].message.content

# Probar con diferentes valores de top_p
top_p_values = [0.1, 0.3, 0.7, 1.0]
for value in top_p_values:
    print(f"Top P: {value}\n")
    print(call_openai_with_top_p(value))
    print("\n" + "-"*80 + "\n")


Top P: 0.1

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels.
2. **Weight Management

--------------------------------------------------------------------------------

Top P: 0.3

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels.
2. **Weight Management

--------------------------------------------------------------------------------

Top P: 0.7

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1.

## Exercise 2: Frequency Penalty Exploration
### Description:
- This exercise will explore how the 'frequency_penalty' parameter affects the model's tendency to repeat itself.

### Task:
- Generate completions using different frequency_penalty values and note the diversity in responses.



In [17]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": "Tell me a fun fact about space."}],
        max_tokens=60,
        frequency_penalty=frequency_penalty
    )
    
    # Retornar la respuesta generada
    return response.choices[0].message.content

# Probar con diferentes valores de frequency_penalty
penalties = [-1.0, 0.0, 1.0, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Frequency Penalty: -1.0

A fun fact about space is that there is some water floating in space! Scientists have discovered massive cloud formations of water vapor in space, with one of the largest being around a black hole called APM 08279+5255. This cloud holds about 140 trillion times the amount of water in

--------------------------------------------------------------------------------

Frequency Penalty: 0.0

A fun fact about space is that a day on Venus is longer than a year on Venus! It takes Venus about 243 Earth days to rotate once on its axis (a single day), but it only takes about 225 Earth days to complete one orbit around the Sun (a year). This unusual

--------------------------------------------------------------------------------

Frequency Penalty: 1.0

Sure! Did you know that there are more stars in the universe than grains of sand on all the beaches on Earth? It's estimated that there are about 100 billion to 200 billion galaxies, each containing millions or even tril

## Exercise 3: Multi-Completion with 'n' Parameter
### Description:
- This exercise will help students generate multiple completions for the same prompt and compare the results.

### Task:
- Generate 3 completions for the same prompt by adjusting the 'n' parameter.


In [19]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": "Describe the benefits of regular exercise."}],
    max_tokens=60,
    n=3
)

# Iteramos sobre las opciones generadas para imprimir las respuestas
for index, c in enumerate(response.choices):
    print(f"Completion {index + 1}: {c.message.content}")
    print("\n" + "-"*80 + "\n")


Completion 1: Regular exercise offers a wide range of benefits for both physical and mental health. Here are some key advantages:

### Physical Health Benefits

1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, enhances circulation, and reduces the risk of heart disease and stroke.

2. **Weight Management

--------------------------------------------------------------------------------

Completion 2: Regular exercise offers a wide range of benefits for both physical and mental health. Here are some key advantages:

### Physical Health Benefits:

1. **Improved Cardiovascular Health**: Exercise strengthens the heart, improves circulation, and enhances the efficiency of the cardiovascular system, reducing the risk of heart disease.



--------------------------------------------------------------------------------

Completion 3: Regular exercise offers a wide range of benefits for both physical and mental health. Here are some key advantages:

### Physical Hea

## Exercise 4: Temperature vs. Deterministic Output
### Description:
- Adjust the 'temperature' parameter to explore the balance between creative and deterministic responses.

### Task:
- Generate completions with different temperature values and analyze the variance in output.


In [20]:
def call_openai_with_temperature(temperature):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": "Suggest a name for a new tech startup."}],
        max_tokens=30,
        temperature=temperature
    )
    return response.choices[0].message.content

# Probar con diferentes valores de temperatura para ver cómo varían las respuestas
temperatures = [0.0, 0.3, 0.7, 1.0]
for temp in temperatures:
    print(f"Temperature: {temp}\n")
    print(call_openai_with_temperature(temp))
    print("\n" + "-"*80 + "\n")


Temperature: 0.0

Sure! Here are some suggestions for a tech startup name:

1. **TechNova**
2. **InnoSphere**
3. **ByteBridge

--------------------------------------------------------------------------------

Temperature: 0.3

Sure! Here are some name suggestions for your tech startup:

1. **TechNova**
2. **InnoVibe**
3. **Byte

--------------------------------------------------------------------------------

Temperature: 0.7

Sure! Here are a few suggestions for a tech startup name:

1. **InnovaSphere**
2. **TechNest**
3. **Quantum

--------------------------------------------------------------------------------

Temperature: 1.0

Sure! Here are some name suggestions for your tech startup:

1. **Techspire**
2. **InnoWave**
3. **Nex

--------------------------------------------------------------------------------

